In [1]:
import numpy as np

source = np.array([0.0, 0.0, 0.0])
direction = np.array([1/np.sqrt(14), 2/np.sqrt(14), 3/np.sqrt(14)])
grid = np.ones((101, 101, 101))
current_voxel = np.array([50, 50, 50])
voxel_size = np.array([1.0, 1.0, 1.0])
    

In [41]:
import numpy as np

def dda_3d(source, direction, grid, original_voxel, voxel_size):

    step = np.zeros(3)
    step[0] = -1 if direction[0] < 0 else 1
    step[1] = -1 if direction[1] < 0 else 1
    step[2] = -1 if direction[2] < 0 else 1

    current_voxel = np.copy(original_voxel)
    
    t = np.zeros(3)
    t[0] = (voxel_size[0] / 2) / direction[0]
    t[1] = (voxel_size[1] / 2) / direction[1]
    t[2] = (voxel_size[2] / 2) / direction[2]

    delta_t = np.zeros(3)
    delta_t[0] = voxel_size[0] / direction[0]
    delta_t[1] = voxel_size[1] / direction[1]
    delta_t[2] = voxel_size[2] / direction[2]

    xmax, ymax, zmax = grid.shape

    voxels_traversed = []
    intersection_t_values = []

    while (current_voxel[0] >= 0 and current_voxel[0] < xmax and
           current_voxel[1] >= 0 and current_voxel[1] < ymax and
           current_voxel[2] >= 0 and current_voxel[2] < zmax):

        voxels_traversed.append(np.copy(current_voxel))
        if t[0] < t[1]:
            if t[0] < t[2]:
                intersection_t_values.append(t[0])
                t[0] += delta_t[0]
                current_voxel[0] += step[0]
            else:
                intersection_t_values.append(t[2])
                t[2] += delta_t[2]
                current_voxel[2] += step[2]
        else:
            if t[1] < t[2]:
                intersection_t_values.append(t[1])
                t[1] += delta_t[1]
                current_voxel[1] += step[1]
            else:
                intersection_t_values.append(t[2])
                t[2] += delta_t[2]
                current_voxel[2] += step[2]

    return intersection_t_values

In [46]:
%timeit dda_3d(source, direction, grid, current_voxel, voxel_size)

761 µs ± 13.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [45]:
dda_3d(source, direction, grid, current_voxel, voxel_size)

[0.6236095644623235,
 0.9354143466934853,
 1.8708286933869704,
 1.8708286933869707,
 2.806243040080456,
 3.1180478223116177,
 4.365266951236265,
 4.677071733467427,
 5.612486080160912,
 5.612486080160912,
 6.547900426854397,
 6.8597052090855595,
 8.106924338010206,
 8.418729120241368,
 9.354143466934852,
 9.354143466934854,
 10.289557813628338,
 10.601362595859499,
 11.848581724784145,
 12.160386507015309,
 13.095800853708791,
 13.095800853708795,
 14.03121520040228,
 14.343019982633438,
 15.590239111558084,
 15.90204389378925,
 16.837458240482732,
 16.837458240482736,
 17.77287258717622,
 18.08467736940738,
 19.33189649833203,
 19.643701280563192,
 20.579115627256677,
 20.579115627256677,
 21.514529973950165,
 21.826334756181325,
 23.073553885105973,
 23.385358667337137,
 24.320773014030618,
 24.32077301403062,
 25.25618736072411,
 25.56799214295527,
 26.815211271879917,
 27.12701605411108,
 28.06243040080456,
 28.062430400804566,
 28.997844747498053,
 29.309649529729214,
 30.55686865

In [2]:
load_ext Cython

In [3]:
import numpy as np

source = np.array([0.0, 0.0, 0.0], dtype=np.float64)
direction = np.array([0.0, 0.0, -1.0], dtype=np.float64)
grid = np.ones((101, 101, 101), dtype=np.float64)
current_voxel = np.array([50, 50, 50], dtype=np.int32)
voxel_size = np.array([1.0, 1.0, 1.0], dtype=np.float64)


In [4]:
%%cython
import numpy as np
cimport cython
cimport numpy as cnp

def dda_3d_naive(cnp.float64_t[:] source, cnp.float64_t[:] direction, cnp.float64_t[:,:,:] grid, cnp.int32_t[:] current_voxel, cnp.float64_t[:] voxel_size):

    cdef cnp.int32_t step[3]
    step[0] = -1 if direction[0] < 0 else 1
    step[1] = -1 if direction[1] < 0 else 1
    step[2] = -1 if direction[2] < 0 else 1
    
    cdef cnp.int32_t vox[3]
    vox[0] = current_voxel[0]
    vox[1] = current_voxel[1]
    vox[2] = current_voxel[2]

    cdef cnp.float64_t t[3]
    t[0] = (voxel_size[0] / 2) / direction[0]
    t[1] = (voxel_size[1] / 2) / direction[1]
    t[2] = (voxel_size[2] / 2) / direction[2]

    cdef cnp.float64_t delta_t[3]
    delta_t[0] = voxel_size[0] / direction[0]
    delta_t[1] = voxel_size[1] / direction[1]
    delta_t[2] = voxel_size[2] / direction[2]

    cdef cnp.float64_t xmax = grid.shape[0]
    cdef cnp.float64_t ymax = grid.shape[1]
    cdef cnp.float64_t zmax = grid.shape[2]

    cdef list voxels_traversed = []
    cdef list intersection_t_values = []
    
    while (vox[0] >= 0 and vox[0] < xmax and
           vox[1] >= 0 and vox[1] < ymax and
           vox[2] >= 0 and vox[2] < zmax):
        
        voxels_traversed.append(np.copy(vox))
        if t[0] < t[1]:
            if t[0] < t[2]:
                intersection_t_values.append(t[0])
                t[0] += delta_t[0]
                vox[0] = vox[0] + step[0]
            else:
                intersection_t_values.append(t[2])
                t[2] += delta_t[2]
                vox[2] = vox[2] + step[2]
        else:
            if t[1] < t[2]:
                intersection_t_values.append(t[1])
                t[1] += delta_t[1]
                vox[1] = vox[1] + step[1]
            else:
                intersection_t_values.append(t[2])
                t[2] += delta_t[2]
                vox[2] = vox[2] + step[2]

    return intersection_t_values


In [22]:
%timeit dda_3d_naive(source, direction, grid, current_voxel, voxel_size)

228 µs ± 27.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
%%cython

import numpy as np
cimport cython
cimport numpy as cnp
@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
@cython.cdivision(True)

def dda_3d_fast(cnp.float64_t[:] source, cnp.float64_t[:] direction, cnp.float64_t[:,:,:] grid, cnp.int32_t[:] current_voxel, cnp.float64_t[:] voxel_size):

    cdef cnp.int32_t step[3]
    step[0] = -1 if direction[0] < 0 else 1
    step[1] = -1 if direction[1] < 0 else 1
    step[2] = -1 if direction[2] < 0 else 1
    
    cdef cnp.int32_t vox[3]
    vox[0] = current_voxel[0]
    vox[1] = current_voxel[1]
    vox[2] = current_voxel[2]

    cdef cnp.float64_t t[3]
    t[0] = (voxel_size[0] / 2) / direction[0]
    t[1] = (voxel_size[1] / 2) / direction[1]
    t[2] = (voxel_size[2] / 2) / direction[2]

    cdef cnp.float64_t delta_t[3]
    delta_t[0] = voxel_size[0] / direction[0]
    delta_t[1] = voxel_size[1] / direction[1]
    delta_t[2] = voxel_size[2] / direction[2]

    cdef cnp.float64_t xmax = grid.shape[0]
    cdef cnp.float64_t ymax = grid.shape[1]
    cdef cnp.float64_t zmax = grid.shape[2]

    cdef cnp.int32_t voxels_traversed[200][3]
    cdef cnp.float64_t intersection_t_values[200]
    
    cdef cnp.int32_t v_count = 0 
    cdef cnp.int32_t i_count = 0
    
    while (vox[0] >= 0 and vox[0] < xmax and
           vox[1] >= 0 and vox[1] < ymax and
           vox[2] >= 0 and vox[2] < zmax):

        voxels_traversed[v_count][0] = vox[0]
        voxels_traversed[v_count][1] = vox[1]
        voxels_traversed[v_count][2] = vox[2]
        if t[0] < t[1]:
            if t[0] < t[2]:
                intersection_t_values[i_count] = t[0]
                t[0] += delta_t[0]
                vox[0] = vox[0] + step[0]
            else:
                intersection_t_values[i_count] = t[2]
                t[2] += delta_t[2]
                vox[2] = vox[2] + step[2]
        else:
            if t[1] < t[2]:
                intersection_t_values[i_count] = t[1]
                t[1] += delta_t[1]
                vox[1] = vox[1] + step[1]
            else:
                intersection_t_values[i_count] = t[2]
                t[2] += delta_t[2]
                vox[2] = vox[2] + step[2]
        v_count = v_count + 1
        i_count = i_count + 1
    
    return intersection_t_values

In [14]:
%timeit dda_3d_fast(source, direction, grid, current_voxel, voxel_size)

4.17 µs ± 20.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [17]:
dda_3d_fast(source, direction, grid, current_voxel, voxel_size)

[0.6236095644623235,
 0.9354143466934853,
 1.8708286933869704,
 1.8708286933869707,
 2.806243040080456,
 3.1180478223116177,
 4.365266951236265,
 4.677071733467427,
 5.612486080160912,
 5.612486080160912,
 6.547900426854397,
 6.8597052090855595,
 8.106924338010206,
 8.418729120241368,
 9.354143466934852,
 9.354143466934854,
 10.289557813628338,
 10.601362595859499,
 11.848581724784145,
 12.160386507015309,
 13.095800853708791,
 13.095800853708795,
 14.03121520040228,
 14.343019982633438,
 15.590239111558084,
 15.90204389378925,
 16.837458240482732,
 16.837458240482736,
 17.77287258717622,
 18.08467736940738,
 19.33189649833203,
 19.643701280563192,
 20.579115627256677,
 20.579115627256677,
 21.514529973950165,
 21.826334756181325,
 23.073553885105973,
 23.385358667337137,
 24.320773014030618,
 24.32077301403062,
 25.25618736072411,
 25.56799214295527,
 26.815211271879917,
 27.12701605411108,
 28.06243040080456,
 28.062430400804566,
 28.997844747498053,
 29.309649529729214,
 30.55686865

In [19]:
import numpy as np

source = np.array([0.0, 0.0, 0.0], dtype=np.float64)
direction = np.array([0.997859, 0.0, -0.065403], dtype=np.float64)
grid = np.ones((41, 41, 41), dtype=np.float64)
current_voxel = np.array([14, 14, 4], dtype=np.int32)
voxel_size = np.array([1.0, 1.0, 1.0], dtype=np.float64)

In [29]:
%%cython

import numpy as np
cimport cython
cimport numpy as cnp
@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
@cython.cdivision(False)

def dda_3d_fast_mv(cnp.float64_t[:] source, cnp.float64_t[:] direction, cnp.float64_t[:,:,:] grid, cnp.int32_t[:] current_voxel, cnp.float64_t[:] voxel_size):

    cdef cnp.int32_t step[3]
    step[0] = -1 if direction[0] < 0 else 1
    step[1] = -1 if direction[1] < 0 else 1
    step[2] = -1 if direction[2] < 0 else 1
    
    if direction[0] < 0:
        direction[0] = -1 * direction[0]
    if direction[1] < 0:
        direction[1] = -1 * direction[1]
    if direction[2] < 0:
        direction[2] = -1 * direction[2]
    
    cdef cnp.int32_t vox[3]
    vox[0] = current_voxel[0]
    vox[1] = current_voxel[1]
    vox[2] = current_voxel[2]

    cdef cnp.float64_t t[3]
    cdef cnp.float64_t delta_t[3]
    if direction[0] == 0.0:
        t[0] = 1000000000
        delta_t[0] = 1000000000
    else:
        t[0] = (voxel_size[0] / 2) / direction[0]
        delta_t[0] = voxel_size[0] / direction[0]
    if direction[1] == 0.0:
        t[1] = 1000000000
        delta_t[1] = 1000000000
    else:
        t[1] = (voxel_size[1] / 2) / direction[1]
        delta_t[1] = voxel_size[1] / direction[1]
    if direction[2] == 0.0:
        t[2] = 1000000000
        delta_t[2] = 1000000000
    else:
        t[2] = (voxel_size[2] / 2) / direction[2]
        delta_t[2] = voxel_size[2] / direction[2]
    
    cdef cnp.int32_t xmax = grid.shape[0]
    cdef cnp.int32_t ymax = grid.shape[1]
    cdef cnp.int32_t zmax = grid.shape[2]

    voxels_traversed = np.zeros((xmax + ymax + zmax, 3), dtype=np.int32)
    cdef cnp.int32_t [:, :] voxels_traversed_mv = voxels_traversed

    intersection_t_values = np.zeros(xmax + ymax + zmax + 10, dtype=np.float64)
    cdef cnp.float64_t [:] intersection_t_values_mv = intersection_t_values
    
    #cdef cnp.int32_t voxels_traversed[200][3]
    #cdef cnp.float64_t intersection_t_values[200]
    
    cdef cnp.int32_t v_count = 0 
    cdef cnp.int32_t i_count = 0
    
    while (vox[0] >= 0 and vox[0] < xmax and
           vox[1] >= 0 and vox[1] < ymax and
           vox[2] >= 0 and vox[2] < zmax):

        voxels_traversed_mv[v_count][0] = vox[0]
        voxels_traversed_mv[v_count][1] = vox[1]
        voxels_traversed_mv[v_count][2] = vox[2]
        print(t)
        if t[0] < t[1]:
            if t[0] < t[2]:
                intersection_t_values_mv[i_count] = t[0]
                t[0] += delta_t[0]
                vox[0] = vox[0] + step[0]
            else:
                intersection_t_values_mv[i_count] = t[2]
                t[2] += delta_t[2]
                vox[2] = vox[2] + step[2]
        else:
            if t[1] < t[2]:
                intersection_t_values_mv[i_count] = t[1]
                t[1] += delta_t[1]
                vox[1] = vox[1] + step[1]
            else:
                intersection_t_values_mv[i_count] = t[2]
                t[2] += delta_t[2]
                vox[2] = vox[2] + step[2]
        v_count = v_count + 1
        i_count = i_count + 1
    
    return (np.asarray(intersection_t_values_mv[:i_count]), np.asarray(voxels_traversed_mv[:v_count, :]))

In [8]:
%timeit dda_3d_fast_mv(source, direction, grid, current_voxel, voxel_size)

10.9 µs ± 128 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [30]:
intersections, voxels = dda_3d_fast_mv(source, direction, grid, current_voxel, voxel_size)

[0.5010727968580732, 1000000000.0, 7.644909254927144]
[1.5032183905742196, 1000000000.0, 7.644909254927144]
[2.5053639842903657, 1000000000.0, 7.644909254927144]
[3.5075095780065118, 1000000000.0, 7.644909254927144]
[4.509655171722658, 1000000000.0, 7.644909254927144]
[5.511800765438804, 1000000000.0, 7.644909254927144]
[6.51394635915495, 1000000000.0, 7.644909254927144]
[7.516091952871096, 1000000000.0, 7.644909254927144]
[8.518237546587242, 1000000000.0, 7.644909254927144]
[8.518237546587242, 1000000000.0, 22.93472776478143]
[9.52038314030339, 1000000000.0, 22.93472776478143]
[10.522528734019536, 1000000000.0, 22.93472776478143]
[11.524674327735683, 1000000000.0, 22.93472776478143]
[12.52681992145183, 1000000000.0, 22.93472776478143]
[13.528965515167977, 1000000000.0, 22.93472776478143]
[14.531111108884124, 1000000000.0, 22.93472776478143]
[15.533256702600271, 1000000000.0, 22.93472776478143]
[16.535402296316416, 1000000000.0, 22.93472776478143]
[17.537547890032563, 1000000000.0, 22.

In [31]:
intersections

array([ 0.5010728 ,  1.50321839,  2.50536398,  3.50750958,  4.50965517,
        5.51180077,  6.51394636,  7.51609195,  7.64490925,  8.51823755,
        9.52038314, 10.52252873, 11.52467433, 12.52681992, 13.52896552,
       14.53111111, 15.5332567 , 16.5354023 , 17.53754789, 18.53969348,
       19.54183908, 20.54398467, 21.54613026, 22.54827586, 22.93472776,
       23.55042145, 24.55256705, 25.55471264, 26.55685823])

In [32]:
voxels

array([[14, 14,  4],
       [15, 14,  4],
       [16, 14,  4],
       [17, 14,  4],
       [18, 14,  4],
       [19, 14,  4],
       [20, 14,  4],
       [21, 14,  4],
       [22, 14,  4],
       [22, 14,  3],
       [23, 14,  3],
       [24, 14,  3],
       [25, 14,  3],
       [26, 14,  3],
       [27, 14,  3],
       [28, 14,  3],
       [29, 14,  3],
       [30, 14,  3],
       [31, 14,  3],
       [32, 14,  3],
       [33, 14,  3],
       [34, 14,  3],
       [35, 14,  3],
       [36, 14,  3],
       [37, 14,  3],
       [37, 14,  2],
       [38, 14,  2],
       [39, 14,  2],
       [40, 14,  2]], dtype=int32)

In [33]:
%%cython -a

import cython
cimport numpy as cnp
from conehead.vector cimport vector, vector_init, vector_append, vector_get, vector_set, vector_free

cdef struct result:
    vector* intersection_t_values
    vector* voxels_traversed

@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
@cython.cdivision(True)
cdef result dda_3d_c(cnp.float64_t* direction, cnp.int32_t* grid_shape, cnp.int32_t* first_voxel, cnp.float64_t* voxel_size) nogil:
    """ Calculate the intersection points of a ray with a voxel grid, using a
    3D DDA algorithm.

    Parameters
    ----------
    direction : ndarray
        direction vector of the ray
    grid : ndarray
        Shape of 3D voxel grid
    first_voxel : ndarray
        Index of ray source voxel
    voxel_size : ndarray
        Size of voxel dimensions

    Returns
    -------
    intersection_t_values_mv : ndarray
        Array of t values corresponding to voxel boundary intersections
    voxels_traversed_mv : ndarray
        List of voxel indices intersected by ray
    """
    cdef cnp.int32_t step[3]
    step[0] = -1 if direction[0] < 0 else 1
    step[1] = -1 if direction[1] < 0 else 1
    step[2] = -1 if direction[2] < 0 else 1

    if direction[0] < 0:
        direction[0] = -1 * direction[0]
    if direction[1] < 0:
        direction[1] = -1 * direction[1]
    if direction[2] < 0:
        direction[2] = -1 * direction[2]

    cdef cnp.int32_t current_voxel[3]
    current_voxel[0] = first_voxel[0]
    current_voxel[1] = first_voxel[1]
    current_voxel[2] = first_voxel[2]

    cdef cnp.float64_t t[3]
    cdef cnp.float64_t delta_t[3]
    cdef cnp.float64_t big_number = 1000000000
    if direction[0] == 0.0:
        t[0] = big_number
        delta_t[0] = big_number
    else:
        t[0] = (voxel_size[0] / 2) / direction[0]
        delta_t[0] = voxel_size[0] / direction[0]
    if direction[1] == 0.0:
        t[1] = big_number
        delta_t[1] = big_number
    else:
        t[1] = (voxel_size[1] / 2) / direction[1]
        delta_t[1] = voxel_size[1] / direction[1]
    if direction[2] == 0.0:
        t[2] = big_number
        delta_t[2] = big_number
    else:
        t[2] = (voxel_size[2] / 2) / direction[2]
        delta_t[2] = voxel_size[2] / direction[2]

    cdef cnp.int32_t xmax = grid_shape[0]
    cdef cnp.int32_t ymax = grid_shape[1]
    cdef cnp.int32_t zmax = grid_shape[2]

    cdef vector* voxels_traversed
    vector_init(voxels_traversed)
    
    cdef vector* intersection_t_values
    vector_init(intersection_t_values)

    cdef cnp.int32_t v_count = 0
    cdef cnp.int32_t i_count = 0

    while (current_voxel[0] >= 0 and current_voxel[0] < xmax and
           current_voxel[1] >= 0 and current_voxel[1] < ymax and
           current_voxel[2] >= 0 and current_voxel[2] < zmax):
        
        vector_append(voxels_traversed, copy_v(current_voxel))
        
        if t[0] < t[1]:
            if t[0] < t[2]:
                vector_append(intersection_t_values, copy_t(t[0]))
                t[0] += delta_t[0]
                current_voxel[0] = current_voxel[0] + step[0]
            else:
                vector_append(intersection_t_values, copy_t(t[1]))
                t[2] += delta_t[2]
                current_voxel[2] = current_voxel[2] + step[2]
        else:
            if t[1] < t[2]:
                vector_append(intersection_t_values, copy_t(t[1]))
                t[1] += delta_t[1]
                current_voxel[1] = current_voxel[1] + step[1]
            else:
                vector_append(intersection_t_values, copy_t(t[2]))
                t[2] += delta_t[2]
                current_voxel[2] = current_voxel[2] + step[2]
        
    cdef result r
    r.intersection_t_values = intersection_t_values
    r.voxels_traversed = voxels_traversed

    return r

cdef cnp.float64_t* copy_t(cnp.float64_t t) nogil:
    
    cdef cnp.float64_t new_t
    new_t = t
    return &new_t

cdef cnp.int32_t* copy_v(cnp.int32_t* v) nogil:
    
    cdef cnp.int32_t new_v[3]
    new_v[0] = v[0]
    new_v[1] = v[1]
    new_v[2] = v[2]

    return new_v
